In [1]:
import os
from scripts.MicroBrix import MicroBrix
import geopandas as gpd
import json
from numpy import mean
import random
import requests

In [2]:
# def get_data_from_server():
#     dataset_server_address = "http://indicator-api:9875"
#     response = requests.get(f'{dataset_server_address}/get_layers_and_numeric/')
#     data = response.json()
#     print(data)
#     return data['layers'], data['numeric']

In [3]:
def get_data():
    import matplotlib.pyplot as plt
    layers = []
    layer_id = 'id_de_la_capa'
    layer_type = 'geojsonbase'
    layer = gpd.read_file("ruta_al_archivo")

    # Parametros para el color del plot
    colormap_name = 'jet' # buscar color maps en matplotlib
    colormap = plt.get_cmap(colormap_name)
    value_column = 'columna_con_valores'
    min_value = 0 # valor minimo de la escala
    max_value = 100 # valor maximo de la escala
    if value_column is None:
        raise ValueError("Value column must be set before calculating colors.")
    norm = plt.Normalize(vmin=min_value, vmax=max_value)
    layer['color'] = layer[value_column].apply(lambda x: colormap(norm(x))[:3])
    layer['color'] = layer['color'].apply(lambda rgb: tuple(int(c * 255) for c in rgb))
    layer['fill'] = layer['color'].apply(lambda x: '#%02x%02x%02x' % (x[0], x[1], x[2]))

    features = json.loads(layer.to_json())
    layers.append({"id": layer_id, "type": layer_type, "data": features, "properties": {"filled": True}})

    return {
        "layers": layers,
        "numeric": []
    }

def serve_indicators(geogrid, geogrid_data):
    return get_data()

In [4]:
connection = MicroBrix(
    table_name='costanera_conce',
    module_function=serve_indicators,
    keep_updating=True,
    quietly=True,
)

In [ ]:
connection.listen()

## Opened connection
{'layers': [{'id': 'grid_combined', 'type': 'geojsonbase', 'data': {'type': 'FeatureCollection', 'features': [{'id': '0', 'type': 'Feature', 'properties': {'index': 0, 'elevacion_': 10.0, 'altura_edi': 0.0, 'altura_tot': 10.0, 'altura_t_1': 1, 'LEGOS': 1}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-73.0792605906241, -36.82272982963954], [-73.07867046764498, -36.82248470905775], [-73.07836557669587, -36.82295913937228], [-73.07895569668206, -36.82320425502967], [-73.0792605906241, -36.82272982963954]]]}}, {'id': '1', 'type': 'Feature', 'properties': {'index': 1, 'elevacion_': 10.0, 'altura_edi': 0.0, 'altura_tot': 10.0, 'altura_t_1': 1, 'LEGOS': 1}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-73.07867046764498, -36.82248470905775], [-73.07808034044703, -36.822239590090255], [-73.0777754524909, -36.822714025329184], [-73.07836557669587, -36.82295913937228], [-73.07867046764498, -36.82248470905775]]]}}, {'id': '2', 'type': 'Feature', 'properties': {'ind

In [8]:
connection.stop()

In [8]:
connection.ws.close()